# Binary Search

**Purpose:** In this article we will explore binary search in ocaml

In [10]:
#require "jupyter.notebook" ;;
(* #require "jupyter-archimedes" *)


/Users/isaacflath/.opam/4.13.1/lib/bytes: added to search path
/Users/isaacflath/.opam/4.13.1/lib/base64: added to search path
/Users/isaacflath/.opam/4.13.1/lib/base64/base64.cma: loaded
/Users/isaacflath/.opam/4.13.1/lib/easy-format: added to search path
/Users/isaacflath/.opam/4.13.1/lib/easy-format/easy_format.cma: loaded
/Users/isaacflath/.opam/4.13.1/lib/biniou: added to search path
/Users/isaacflath/.opam/4.13.1/lib/biniou/biniou.cma: loaded
/Users/isaacflath/.opam/4.13.1/lib/yojson: added to search path
/Users/isaacflath/.opam/4.13.1/lib/yojson/yojson.cma: loaded
/Users/isaacflath/.opam/4.13.1/lib/ppx_yojson_conv_lib: added to search path
/Users/isaacflath/.opam/4.13.1/lib/ppx_yojson_conv_lib/ppx_yojson_conv_lib.cma: loaded
/Users/isaacflath/.opam/4.13.1/lib/uuidm: added to search path
/Users/isaacflath/.opam/4.13.1/lib/uuidm/uuidm.cma: loaded
/Users/isaacflath/.opam/4.13.1/lib/jupyter: added to search path
/Users/isaacflath/.opam/4.13.1/lib/jupyter/jupyter.cma: loaded
/Users/i

# Problem Setup

For this problem we will be fed a direction, but we won't know the exact coordinate.  Here we create the functions that given a current location and a target location gives us the direction (ie DR, or D)

In [24]:
exception Damn of string

let get_width_dir bw cw = 
  match compare cw bw  with
    |0 -> ""|1 -> "L"|(-1) -> "R"| _ -> raise (Damn "error")

let get_height_dir bh ch = 
  match compare ch bh with
    |0 -> "" |1 -> "U" |(-1) -> "D"| _ -> raise (Damn "error")

let get_direction currentloc bombloc = 
  let bw,bh = bombloc in
  let ((cw,ch),(_,_,_,_)) = currentloc in

  (get_height_dir bh ch) ^ (get_width_dir bw cw)

exception Damn of string


val get_width_dir : 'a -> 'a -> string = <fun>


val get_height_dir : 'a -> 'a -> string = <fun>


val get_direction : ('a * 'b) * ('c * 'd * 'e * 'f) -> 'a * 'b -> string =
  <fun>


# Step Function

Next we need to write out step function.  Every step is to the center of the available board, and we shrink the available board with each step as we are given the location.

In [12]:
let take_step currentloc dir =
  let ((cw,ch),(w0,w1,h0,h1)) = currentloc in
  match dir with 
        |"L" -> (((w0+cw)/2,ch),(w0,cw,ch,ch))
        |"R" -> (((cw+w1)/2,ch),(cw,w1,ch,ch))
        |"U" -> ((cw,(h0+ch)/2),(cw,cw,h0,ch))
        |"D" -> ((cw,(ch+h1)/2),(cw,cw,ch,h1))
        |"UL" -> (((w0+cw)/2,(h0+ch)/2),(w0,cw,h0,ch))
        |"UR" -> (((cw+w1)/2,(w0+ch)/2),(cw,w1,h0,ch))
        |"DL" -> (((w0+cw)/2,(ch+h1)/2),(w0,cw,ch,h1))
        |"DR" -> (((cw+w1)/2,(ch+h1)/2),(cw,w1,ch,h1))
        |_ -> raise (Damn "error")

val take_step :
  (int * int) * (int * int * int * int) ->
  string -> (int * int) * (int * int * int * int) = <fun>


# The Test Loop

We will create a test loop that will iterate through until it finds a solution, with some printing so we can see what it's doing.

In [13]:
let print_step bl cl bd =
    let ((ow,oh),(bw1,bw2,bh1,bh2)) = cl in
    let bw, bh = bl in
    print_endline ("Board: " ^ string_of_int bw1 ^ "x"^ string_of_int bw2 ^ "x"^ string_of_int bh1 ^ "x"^ string_of_int bh2 ^ "  |  " ^
              "Bomb: " ^ string_of_int bw ^ "x" ^ string_of_int bh ^ "  |  " ^ 
              "Loc: " ^ string_of_int ow ^ "x" ^ string_of_int oh ^ "  |  " ^ 
              "Direction: " ^ bd);

val print_step :
  int * int -> (int * int) * (int * int * int * int) -> string -> unit =
  <fun>


In [14]:
let rec loop loc acc printf bl=
    let bombdir = get_direction loc bl in
    printf bl loc bombdir;

    let out = take_step loc bombdir in
    let ((ow,oh),(bw1,bw2,bh1,bh2)) = out in


    if (ow,oh)=(bl) then "success" 
    else if acc > 200 then "200 iters"
    else loop out (acc+1) printf bl

val loop :
  (int * int) * (int * int * int * int) ->
  int ->
  (int * int -> (int * int) * (int * int * int * int) -> string -> 'a) ->
  int * int -> string = <fun>


# The Expirament

We can see that even when run with a pretty large random size and locations setup, we solve the problem in relatively few steps.

In [25]:
let sz = 1_000_000_000
let w,h = (Random.int sz,Random.int sz)
let current_loc = (Random.int w,Random.int h)
let bomb_loc = (Random.int w,Random.int h)
let board = (0,w,0,h)
let current_loc = (current_loc,board)

val sz : int = 1000000000


val w : int = 735866460
val h : int = 433353897


val current_loc : int * int = (501997763, 64262183)


val bomb_loc : int * int = (352563599, 390644720)


val board : int * int * int * int = (0, 735866460, 0, 433353897)


val current_loc : (int * int) * (int * int * int * int) =
  ((501997763, 64262183), (0, 735866460, 0, 433353897))


In [23]:
loop current_loc 0 print_step bomb_loc

Board: 0x179793212x0x425739649  |  Bomb: 95278349x183780708  |  Loc: 59186575x204244755  |  Direction: UR
Board: 59186575x179793212x0x204244755  |  Bomb: 95278349x183780708  |  Loc: 119489893x102122377  |  Direction: DL
Board: 59186575x119489893x102122377x204244755  |  Bomb: 95278349x183780708  |  Loc: 89338234x153183566  |  Direction: DR
Board: 89338234x119489893x153183566x204244755  |  Bomb: 95278349x183780708  |  Loc: 104414063x178714160  |  Direction: DL
Board: 89338234x104414063x178714160x204244755  |  Bomb: 95278349x183780708  |  Loc: 96876148x191479457  |  Direction: UL
Board: 89338234x96876148x178714160x191479457  |  Bomb: 95278349x183780708  |  Loc: 93107191x185096808  |  Direction: UR
Board: 93107191x96876148x178714160x185096808  |  Bomb: 95278349x183780708  |  Loc: 94991669x137217521  |  Direction: DR
Board: 94991669x96876148x137217521x185096808  |  Bomb: 95278349x183780708  |  Loc: 95933908x161157164  |  Direction: DL
Board: 94991669x95933908x161157164x185096808  |  Bomb: 9

- : string = "success"
